In [2]:
import pandas as pd
df = pd.read_csv('SampleDataset.csv', encoding='latin-1')

In [3]:
df.head()

,Sno,TECHNOLOGY,ORIGINAL/SYNONYM,QUESTION,SOLUTION
0,1,PowerBI,Original,Issue with Power bi date slicer,If you are experiencing an issue with the date...
1,2,PowerBI,Synonym,What are some common issues that can arise wit...,same as above
2,3,PowerBI,Synonym,How do I resolve issues with the date slicer i...,Same as above
3,4,PowerBI,Synonym,I'm having trouble getting the date slicer in ...,Same as above
4,5,PowerBI,Synonym,What could be causing the date slicer in Power...,Same as above


In [6]:
df['Sentence'] = df['TECHNOLOGY'] + " - " + df['QUESTION']
df.head()


,Sno,TECHNOLOGY,ORIGINAL/SYNONYM,QUESTION,SOLUTION,Sentence
0,1,PowerBI,Original,Issue with Power bi date slicer,If you are experiencing an issue with the date...,PowerBI - Issue with Power bi date slicer
1,2,PowerBI,Synonym,What are some common issues that can arise wit...,same as above,PowerBI - What are some common issues that can...
2,3,PowerBI,Synonym,How do I resolve issues with the date slicer i...,Same as above,PowerBI - How do I resolve issues with the dat...
3,4,PowerBI,Synonym,I'm having trouble getting the date slicer in ...,Same as above,PowerBI - I'm having trouble getting the date ...
4,5,PowerBI,Synonym,What could be causing the date slicer in Power...,Same as above,PowerBI - What could be causing the date slice...


In [9]:
sentences = []
for row in df['Sentence']:
    sentences.append(row)
sentences[1:8]

['PowerBI - What are some common issues that can arise with the date slicer in Power BI, and how can I troubleshoot them?',
 'PowerBI - How do I resolve issues with the date slicer in Power BI when it is not \nfiltering data correctly or not displaying all available dates?',
 "PowerBI - I'm having trouble getting the date slicer in Power BI to work properly. Are \nthere any specific settings or configurations I should check to resolve the issue?",
 'PowerBI - What could be causing the date slicer in Power BI to malfunction, and what \nsteps can I take to fix it?',
 'PowerBI - Can you provide any guidance on troubleshooting problems with the date \nslicer in Power BI, particularly when it comes to issues with filtering or interaction with other visuals and filters?',
 'PowerBI - The date slicer is not filtering data correctly',
 'PowerBI - What should I do if my Power BI date slicer is not filtering data correctly?']

In [11]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
embeddings = model.encode(sentences)
print(embeddings)

[[-0.05455767  0.07401679  0.00639784 ... -0.04954587 -0.05768153
  -0.02505915]
 [-0.04970559  0.03084412  0.01144503 ... -0.0610906  -0.07576085
  -0.01194992]
 [-0.0043903   0.06759154  0.003647   ... -0.05403585 -0.07599874
   0.00057957]
 ...
 [-0.01956022 -0.0618501  -0.05478379 ... -0.03913625  0.02141253
  -0.06274448]
 [-0.00899356 -0.08685962 -0.05075131 ... -0.04794435  0.02407048
  -0.0740986 ]
 [-0.03058353 -0.07275827 -0.0599315  ... -0.06807294  0.05030227
  -0.06599381]]
